In [1]:
# https://datanoon.com/blog/loading_data_rest_api_to_spark/

In [2]:
import requests
import json
import time
import pandas as pd

In [3]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.session import SparkSession

# Create a local StreamingContext with batch interval of 1 seconds
conf = SparkConf().setAppName("app1").setMaster("local")
sc = SparkContext(conf=conf)
ssc = StreamingContext(sc, 1)

In [4]:
def get_weather():
    url = f"https://api.openweathermap.org/data/2.5/weather?q=barcelona&appid=24b234cb23d2bddfc71ee6db0e1b1d6c"
    response = requests.get(url)
    return response

In [5]:
# Create the queue through which RDDs can be pushed to
# a QueueInputDStream
rddQueue = []
for i in range(6):
    time.sleep(10)
    weather = get_weather()
    rddQueue += [ssc.sparkContext.parallelize([weather.text])]

In [6]:
inputStream = ssc.queueStream(rddQueue)
inputStream.saveAsTextFiles('streaming_weather')

ssc.start()
time.sleep(6)
ssc.stop(stopSparkContext=True, stopGraceFully=True) 

In [11]:
for weather in rddQueue:
    print(weather)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:262
ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:262
ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:262
ParallelCollectionRDD[3] at readRDDFromFile at PythonRDD.scala:262
ParallelCollectionRDD[4] at readRDDFromFile at PythonRDD.scala:262
ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:262
